# 获取数据

In [1]:
from aliyun.log import *
import time
from datetime import datetime


In [2]:
# 配置AccessKey、服务入口、Project名称、Logstore名称等相关信息。
# AccessKeyId, AccessKeySecret, ProjectName 可以在竞赛详情下的数据页签获取
accessKeyId = "LTAI5tNksVRpmzu8BJnrCbaK"
accessKey = "ULTVXyesWOaTEEwnALbQCNbx3z1vxo"
project_name = "aiops-2022-015"
# 日志服务的域名
endpoint = "cn-chengdu.log.aliyuncs.com"
# Logstore名称, 本次比赛数据包含4中维度, 分别为:
# kpi-12ad198e9103f4878018757d0cd589e2
# metric-12ad198e9103f4878018757d0cd589e2
# trace-12ad198e9103f4878018757d0cd589e2
# log-12ad198e9103f4878018757d0cd589e2
# 以kpi数据为例:
logstore_name = "kpi-12ad198e9103f4878018757d0cd589e2"


In [3]:
# 创建日志服务Client
client = LogClient(endpoint, accessKeyId, accessKey)
print("基础资源初始化")

基础资源初始化


## 通过PullData获取

In [4]:
# 获取当前Project/LogStore下有多少个Shard
res = client.list_shards(project_name, logstore_name)
res


In [5]:
shards_info = res.get_shards_info()
shards = []
for item in shards_info:
    shards.append(item["shardID"])
shards


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### 持续获取LogStore中的数据

In [6]:
single_fetch_max_size = 1000
def sample_pull_logs_batch_continue(shard_id, begin_cursor, compress=False):
    cursor = begin_cursor
    res = client.pull_logs(project_name, logstore_name, shard_id,
                           cursor, single_fetch_max_size, compress=compress)
    n_fetch = res.get_log_count()
    next_cursor = res.get_next_cursor()
    if n_fetch > 0:
        log_groups = res.get_loggroup_json_list()
        return log_groups, next_cursor
    return [], next_cursor


In [7]:
from_date_string = "2022-04-13 08:00:00"
from_date = datetime.fromisoformat(from_date_string)
from_time = int(time.mktime(from_date.timetuple()))
shard_info_map = {}
for shard_id in shards:
    begin_cursor_res = client.get_cursor(
        project_name, logstore_name, shard_id, from_time)
    begin_cursor = begin_cursor_res.get_cursor()
    shard_info_map[shard_id] = {
        "cursor": begin_cursor
    }

idx = 0
while True:
    idx += 1
    batch_total_log_groups = []
    for shard_id in shard_info_map.keys():
        shard_info = shard_info_map[shard_id]
        begin_cursor = shard_info["cursor"]
        log_groups, next_cursor = sample_pull_logs_batch_continue(
            shard_id, begin_cursor)
        batch_total_log_groups.extend(log_groups)
        shard_info["cursor"] = next_cursor
        shard_info_map[shard_id] = shard_info

    # 这里可以统一处理当前这个批次获取的LogGroup
    print("fetch log group size is {}".format(len(batch_total_log_groups)))
    for log_group in batch_total_log_groups:
        raw_logs = log_group["logs"]
        print(raw_logs[0])
    # 处理后，将数据对象清空
    batch_total_log_groups = []
    # print(shard_info_map)
    if idx == 2:
        break


KeyboardInterrupt: 

## 通过SQL/Query获取

In [9]:
query = '''* | select * from log limit 10 '''
# query = '''* | select __time__ as time, "cmdb_id" as cmdb_id, "value" as value, "kpi_name" as kpi_name from log limit 1000'''


In [10]:
# start_time: unixtime时间戳，单位为秒，且要按照分钟对齐
# end_time: unixtime时间戳，单位为秒，且要按照分钟对齐
def sample_get_logs(start_time, end_time,query):
    if start_time % 60 != 0 or end_time % 60 != 0:
        print("start_time and end_time should be align by minute")
        return None

    request = GetLogsRequest(project_name, logstore_name,
                             start_time, end_time, query=query)
    response = client.get_logs(request)
    raw_datas = []
    for log in response.get_logs():
        raw_datas.append(log.contents)
    return raw_datas


In [11]:
from_date_string = "2022-03-25 18:33:00"
from_date = datetime.fromisoformat(from_date_string)
from_time = int(time.mktime(from_date.timetuple()))
step_interval = 60
query_raw_datas = sample_get_logs(from_time, from_time+step_interval, query)
if query_raw_datas is not None:
    print(query_raw_datas[0])


KeyboardInterrupt: 